# Add labels to DB
For every _valid_ experiment **from the validation data**, add labels to a separate table.
We use the following labels:
- scaled MS responses for all products
- binary outcome for all products
- major product (A, B, C, or no_product)

We use the scaling factors obtained on the ML data set (2023-12-20).

In [1]:
from typing import Tuple, Union
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.util.db_utils import SynFermDatabaseConnection
from src.definitions import DATA_DIR

In [2]:
con = SynFermDatabaseConnection()

In [3]:
# get only valid experiments from DB
data = con.con.execute("SELECT * FROM experiments WHERE exp_nr BETWEEN 100 AND 101 AND (valid NOT LIKE '%ERROR%' OR valid IS NULL)").fetchall()
df = pd.DataFrame(data, columns=[c[1] for c in con.con.execute("PRAGMA table_info(experiments)").fetchall()])
len(df)

744

In [4]:
# load scaling factors
scaling_factors = pd.read_csv(DATA_DIR / "scaling-factors_2023-12-20.csv").set_index("product_type")

In [7]:
# apply scaling to all products
for s in "ABCDEFGH":
    df[f'scaled_{s}'] = df[f'product_{s}_lcms_ratio'] / scaling_factors.loc["A", "factor"]

In [8]:
# add binary outcome: 1 if product was formed, 0 if not
df["binary_A"] = (df["scaled_A"] > 0).astype(int)
df["binary_B"] = (df["scaled_B"] > 0).astype(int)
df["binary_C"] = (df["scaled_C"] > 0).astype(int)
df["binary_D"] = (df["scaled_D"] > 0).astype(int)
df["binary_E"] = (df["scaled_E"] > 0).astype(int)
df["binary_F"] = (df["scaled_F"] > 0).astype(int)
df["binary_G"] = (df["scaled_G"] > 0).astype(int)
df["binary_H"] = (df["scaled_H"] > 0).astype(float)
df.loc[df["scaled_H"].isna(), "binary_H"] = pd.NA

In [9]:
df.describe()

,id,exp_nr,plate_nr,product_A_lcms_ratio,product_B_lcms_ratio,product_C_lcms_ratio,product_D_lcms_ratio,product_E_lcms_ratio,product_F_lcms_ratio,product_G_lcms_ratio,...,scaled_G,scaled_H,binary_A,binary_B,binary_C,binary_D,binary_E,binary_F,binary_G,binary_H
count,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,...,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000
mean,89731.743280,100.797043,1.868280,0.139469,0.084224,0.035168,0.217315,0.111825,0.164747,0.031888,...,0.207016,0.172370,0.974462,0.672043,0.404570,0.908602,0.885753,0.983871,0.955645,0.602151
std,447.363772,0.402471,0.817944,0.215757,0.129795,0.175661,0.461961,0.248503,0.183046,0.063950,...,0.415158,0.809652,0.157857,0.469785,0.491139,0.288368,0.318325,0.126057,0.206021,0.489783
min,88994.000000,100.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,89389.750000,101.000000,1.000000,0.008782,0.000000,0.000000,0.003657,0.001946,0.033504,0.003199,...,0.020769,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,89777.500000,101.000000,2.000000,0.061946,0.026269,0.000000,0.035010,0.026797,0.093457,0.008425,...,0.054694,0.007167,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,89974.250000,101.000000,3.000000,0.177627,0.119322,0.002697,0.175026,0.107666,0.242496,0.027747,...,0.180133,0.044275,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,91219.000000,101.000000,3.000000,1.644593,0.934310,2.318021,3.177112,2.943058,1.138627,0.466389,...,3.027781,7.928279,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# assign the main product
df["main_product"] = df[["scaled_A", "scaled_B", "scaled_C"]].idxmax(axis=1).str.replace("scaled_", "")
# are there any reactions where neither A,nor B, nor C appear?
df.loc[df[["scaled_A", "scaled_B", "scaled_C"]].sum(axis=1) == 0, "main_product"] = "no_product"
df

,id,exp_nr,plate_nr,well,lab_journal_number,synthesis_date_unixepoch,initiator,monomer,terminator,initiator_long,...,scaled_H,binary_A,binary_B,binary_C,binary_D,binary_E,binary_F,binary_G,binary_H,main_product
0,88994,100,1,A10,JG405,2023-11-29,None,None,None,Ph031,...,0.307902,1,1,1,1,1,1,1,1.0,A
1,88995,100,1,A11,JG405,2023-11-29,None,None,None,BiAl008,...,0.028144,1,1,1,1,1,1,1,1.0,A
2,88996,100,1,A12,JG405,2023-11-29,None,None,None,BiAl008,...,0.000000,1,1,1,1,1,1,0,0.0,B
3,89007,100,1,A3,JG405,2023-11-29,None,None,None,Ph013,...,0.009608,1,1,1,1,1,1,1,1.0,B
4,89008,100,1,A4,JG405,2023-11-29,None,None,None,Al013,...,0.516732,1,1,1,0,1,1,1,1.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,91197,101,3,O13,JG408,2023-11-29,None,None,None,Ph043,...,0.007491,1,1,1,1,1,1,1,1.0,A
740,91198,101,3,O14,JG408,2023-11-29,None,None,None,Ph043,...,0.058083,1,1,1,1,1,1,1,1.0,A
741,91217,101,3,P13,JG408,2023-11-29,None,None,None,Ph043,...,0.000000,1,1,1,1,1,1,1,0.0,A
742,91218,101,3,P14,JG408,2023-11-29,None,None,None,Ph043,...,0.044159,1,1,0,1,1,1,1,1.0,A


In [11]:
# write to 'labels' table of DB
with con.con:
    con.con.executemany('INSERT INTO labels (experiment_id, scaled_A, scaled_B, scaled_C, scaled_D, scaled_E, scaled_F, scaled_G, scaled_H, binary_A, binary_B, binary_C, binary_D, binary_E, binary_F, binary_G, binary_H, "major_A-C") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);',
                        df[["id", "scaled_A", "scaled_B", "scaled_C", "scaled_D", "scaled_E", "scaled_F", "scaled_G", "scaled_H", "binary_A", "binary_B", "binary_C", "binary_D", "binary_E", "binary_F", "binary_G", "binary_H", "main_product"]].values.tolist()
                        )

In [20]:
df["binary_H"].mean()

0.6021505376344086